### Notes: If ImportError occurs, it's probably due to the huggingface-hub. 
> pip install huggingface-hub==0.25.0

## Import

In [2]:
import torch
from transformers import pipeline

from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline, BitsAndBytesConfig, AutoModelForCausalLM
import os

DEVICE = "cuda:0" if torch.cuda.is_available() else "CPU"
DEVICE = "cpu"
HUGGING_FACE_TOKEN = os.environ.get('HUGGING_FACE_TOKEN') #in terminal: export HUGGING_FACE_TOKEN="YOUR_TOKEN"


## Load Model

In [3]:
'''
Possible Models:
- meta-llama/Llama-3.2-1B-Instruct
- meta-llama/Llama-3.2-3B-Instruct
- meta-llama/Llama-3.2-11B-Vision-Instruct
'''
model_id = "meta-llama/Llama-3.2-3B-Instruct" 

# Quanitisize your model dtype (for sparsity)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Set token using ENV variable
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGING_FACE_TOKEN)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=HUGGING_FACE_TOKEN,
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt = f"HI!"
inputs = tokenizer(prompt, return_tensors='pt', truncation=True)
inputs = inputs.to('cpu')  # Ensure inputs are on CPU

output = model.generate(
    **inputs,
    max_new_tokens=50,
    num_beams=1,
    do_sample=False,
    temperature=1
)

answer = tokenizer.decode(output[0], skip_special_tokens=True)

print(answer)

/home/dongkyu/miniconda3/envs/lm/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/dongkyu/miniconda3/envs/lm/lib/python3.12/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


HI! I'm so excited to share my latest project with you all! I've been working on a new line of eco-friendly, cruelty-free, and vegan-friendly makeup products, and I just can't wait to show you all the gorgeous shades and formulas I


### Knowledge Bases

In [5]:
#pdf base
pdf_base_path = '/home/dongkyu/RAG/knowledge'
loader = PyPDFDirectoryLoader(pdf_base_path) #The Knowledge Base Folder
pdf_docs = loader.load()
len(pdf_docs) 

13

In [6]:
#knowledge base
knowledge_base_path= "/home/dongkyu/exported_docs"
txt_loader = DirectoryLoader(knowledge_base_path, glob="**/*.txt")
docs = txt_loader.load()
len(docs)

442

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)


3905

In [8]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-base",
    model_kwargs={"device": DEVICE}
)

db = Chroma.from_documents(texts, embeddings, persist_directory="db")


load INSTRUCTOR_Transformer
max_seq_length  512


### RAG

In [9]:
DEFAULT_SYSTEM_PROMPT = """
Based on the information in this document provided in context, answer the question as accurately as possible in 1 or 2 lines. If the information is not in the context,
respond with "I don't know" or a similar acknowledgment that the answer is not available.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. Do not provide commentary or elaboration more than 1 or 2 lines.?"

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [10]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

llm = HuggingFacePipeline(pipeline=text_pipeline)

/tmp/ipykernel_992105/3144239959.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline)


In [12]:
ask = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt},
)

result = ask("Write a sample code for Bouncy Bolson")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 

```cpp
#include <ros/ros.h>
#include <std_msgs/String.h>

class FibonacciActionServer : public rclcpp::Node {
public:
    explicit FibonacciActionServer(const rclcpp::NodeOptions & options = rclcpp::NodeOptions()) 
        : Node("bouncy_bolson", options), loop_rate(10.0) {}

    void handle_goal(const std_msgs::String::ConstPtr & msg) override {
        // Handle goal message here.
        std::cout << "Goal received: " << msg->data << std::endl;
    }

    void handle_cancel() override {
        // Handle cancel request here.
        std::cout << "Cancel requested." << std::endl;
    }

    void handle_accept() override {
        // Accept the goal and start execution.
        std::cout << "Goal accepted." << std::endl;
    }
};

int main(int argc, char * argv[]) {
    ros::init(argc, argv, "bouncy_bolon");
    auto fibonacci_node = std::make_shared<FibonacciActionServer>();
    ros::spin(fibonacci_node);
    return 0;
}
```

Note: The provided C++ code snippet does not implement 